Paper:
https://arxiv.org/pdf/1512.04150.pdf

GitHub:
https://github.com/jacobgil/keras-cam

In [5]:
import cv2
import glob
import os
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras import backend as K
import h5py
from keras.optimizers import SGD

from keras.models import *
from keras.callbacks import *
import keras.backend as K
import cv2
import argparse

def load_mnist():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    return np.expand_dims(X_test, axis=1), to_categorical(y_test)

In [6]:
def global_average_pooling(x):
    return K.mean(x, axis = (2, 3))

def global_average_pooling_shape(input_shape):
    return input_shape[0:2]

def VGG16_convolutions():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(1,None,None)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

#     model.add(ZeroPadding2D((1, 1)))
#     model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
#     model.add(ZeroPadding2D((1, 1)))
#     model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
#     model.add(ZeroPadding2D((1, 1)))
#     model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
#     model.add(MaxPooling2D((2, 2), strides=(2, 2)))

#     model.add(ZeroPadding2D((1, 1)))
#     model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
#     model.add(ZeroPadding2D((1, 1)))
#     model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
#     model.add(ZeroPadding2D((1, 1)))
#     model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
#     model.add(MaxPooling2D((2, 2), strides=(2, 2)))

#     model.add(ZeroPadding2D((1, 1)))
#     model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
#     model.add(ZeroPadding2D((1, 1)))
#     model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
#     model.add(ZeroPadding2D((1, 1)))
#     model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    return model

def get_model():
    model = VGG16_convolutions()

#     model = load_model_weights(model, "vgg16_weights.h5")
    
    model.add(Lambda(global_average_pooling, 
              output_shape=global_average_pooling_shape))
    model.add(Dense(10, activation = 'softmax', init='uniform'))
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
    model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])
    return model

def load_model_weights(model, weights_path):
    print('Loading model.')
    f = h5py.File(weights_path)
    if 'nb_layers' in f.attrs.keys():
        for k in range(f.attrs['nb_layers']):
            if k >= len(model.layers):
                # we don't look at the last (fully-connected) layers in the savefile
                break
            g = f['layer_{}'.format(k)]
            weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
            model.layers[k].set_weights(weights)
            model.layers[k].trainable = False
    f.close()
    print('Model loaded.')
    return model

def get_output_layer(model, layer_name):
    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    layer = layer_dict[layer_name]
    return layer

In [7]:
def train():
        model = get_model()
        X, y = load_mnist()
        print("Training..")
        checkpoint_path="weights.{epoch:02d}-{val_loss:.2f}.hdf5"
        checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto')
        model.fit(X, y, nb_epoch=40, batch_size=32, validation_split=0.2, verbose=1, callbacks=[checkpoint])

def visualize_class_activation_map(model_path, img, output_path):
        model = load_model(model_path)
#         original_img = cv2.imread(img_path, 1)
        original_img = img.reshape(28,28,1)
        width, height, _ = original_img.shape

        #Reshape to the network input shape (3, w, h).
        img = np.array([np.transpose(np.float32(original_img), (2, 0, 1))])
        
        #Get the 512 input weights to the softmax.
        class_weights = model.layers[-1].get_weights()[0]
        final_conv_layer = get_output_layer(model, "conv2_2")
        get_output = K.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
        [conv_outputs, predictions] = get_output([img])
        conv_outputs = conv_outputs[0, :, :, :]

        #Create the class activation map.
        cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[1:3])
        for i, w in enumerate(class_weights[:, 1]):
                cam += w * conv_outputs[i, :, :]
        print("predictions", predictions)
        cam /= np.max(cam)
        cam = cv2.resize(cam, (height, width))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam < 0.2)] = 0
        img = heatmap*0.5 + original_img
        cv2.imwrite(output_path, img)

In [ ]:
train()

C:\Users\Michal\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_1")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Michal\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_2")`
  if sys.path[0] == '':
C:\Users\Michal\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_1")`
  app.launch_new_instance()
C:\Users\Michal\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_2")`
C:\Users\Michal\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="s

Training..


C:\Users\Michal\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 8000 samples, validate on 2000 samples
Epoch 1/40
2464/8000 [========>.....................] - ETA: 3:47 - loss: 2.3167 - acc: 0.1380

In [ ]:
visualize_class_activation_map(model_path="weights.02-0.17.hdf5", img=X[0], output_path='test.jpg')